In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


# url = 'faisalq/bert-base-arabic-wordpiece'
url = 'faisalq/bert-base-arabic-senpiece'
# url = 'faisalq/bert-base-arabic-bbpe'


dataset = load_dataset("arbml/WDC")

df = dataset['train'].to_pandas()


# removing incorrect tags
df2 = df[~df['Entity'].str.contains('ORG', regex=True)]
df2 = df2[~df2['Entity'].str.contains('LOC', regex=True)]
df2 = df2[~df2['Entity'].str.contains('PER', regex=True)]
df2 = df2[~df2['Entity'].str.contains('MISC', regex=True)]
df2 = df2[df2['Entity'] != 'O']

v = df2['Entity'].values
v = list(set(v))
df = df[~df['Entity'].isin(v)]

display(set(df['Entity'].values))

classes_num = len(set(df['Entity']))

display(classes_num)

# display(df)
df['Entity'] = df['Entity'].astype('category')
df['label'] = df['Entity'].cat.codes



sentences = []
labels = []

current_sentence = []
current_labels = []

#convert the df into a format suitable for BERT
for _, row in df.iterrows():
    if row['Word'] != '.':  #check if the word is not a sentence separator (period)
        current_sentence.append(row['Word'])
        current_labels.append(row['label'])
    else:
        sentences.append(current_sentence)
        labels.append(current_labels)        
        current_sentence = []
        current_labels = []


dfx = pd.DataFrame({
    'tokens': sentences,
    'ner_tags': labels
})



dataset = Dataset.from_pandas(dfx)



# unique_classes = set()

# for example in dataset:
#     unique_classes.update(example['ner_tags'])



tokenizer = AutoTokenizer.from_pretrained(url)


model = AutoModelForTokenClassification.from_pretrained(url, 
                                num_labels= classes_num)

max_length = 128


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,
                                 padding="max_length", max_length=max_length)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Subword token
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs



# dataset = dataset['train']
dataset = dataset.train_test_split(test_size=0.2)

dataset_train = dataset['train']
dataset_validation = dataset['test']

# display(dataset_train)
    
dataset_train = dataset_train.map(tokenize_and_align_labels, batched=True)
dataset_validation = dataset_validation.map(tokenize_and_align_labels, batched=True)





2024-02-05 09:39:34.273131: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 09:39:34.295158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 09:39:34.694191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Found cached dataset parquet (/home/ffq/.cache/huggingface/datasets/arbml___parquet/arbml--WDC-c6c8f367119243ea/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}

9

Some weights of BertForTokenClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-senpiece and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/151082 [00:00<?, ? examples/s]

Map:   0%|          | 0/37771 [00:00<?, ? examples/s]

In [2]:
from transformers import TrainingArguments, Trainer
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_predictions = [tokenizer.convert_ids_to_tokens(tp) for tp in true_predictions]
    true_labels = [tokenizer.convert_ids_to_tokens(tl) for tl in true_labels]

    # results = classification_report(true_labels, true_predictions)
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        # "report": results,
    }



epochs = 50
save_steps = 10000 #save checkpoint every 10000 steps
batch_size = 256



training_args = TrainingArguments(
    output_dir = 'bert_wp/',
    overwrite_output_dir=True,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_steps = save_steps,
    save_total_limit = 2, 
    fp16=True,
    learning_rate = 5e-5,  # 5e-5 is the default
    # weight_decay=0.01,
    logging_steps = 500, #50_000
    evaluation_strategy = 'steps',
    # logging_dir="logs.txt",
    eval_steps = 500
    
)




# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Step,Training Loss,Validation Loss,Precision,Recall,F1
500,0.287200,0.201947,0.725873,0.715604,0.720702
1000,0.182800,0.177881,0.752735,0.751098,0.751916
1500,0.150500,0.175469,0.752305,0.771862,0.761958
2000,0.126800,0.173398,0.767205,0.773026,0.770104
2500,0.107200,0.177720,0.780784,0.771886,0.776310
3000,0.090900,0.191616,0.770803,0.788229,0.779419
3500,0.075300,0.185361,0.781286,0.780786,0.781036
4000,0.063600,0.194922,0.779829,0.782638,0.781231
4500,0.054600,0.213087,0.772801,0.790918,0.781754
5000,0.047200,0.223187,0.776973,0.789044,0.782962


{'eval_loss': 0.4147399663925171,
 'eval_precision': 0.790105704727895,
 'eval_recall': 0.7998412034851023,
 'eval_f1': 0.7949436480999076,
 'eval_runtime': 27.0471,
 'eval_samples_per_second': 1396.488,
 'eval_steps_per_second': 5.472,
 'epoch': 50.0}

In [3]:
# 795156

In [4]:
len(dataset_train)

151082

In [5]:
len(dataset_validation)

37771